In [134]:
import numpy as np
from scipy.stats import poisson

"""Function to calculate convolution of two arrays"""
def convolve(a, b):
    c = np.zeros(len(a))
    for i in range(len(a)):
        # reverse b
        b_rev = b[::-1]
    
        # dot product of a and b_rev
        c[i] = np.dot(a, b_rev)
    
        # drop the last element of a and b
        a = a[:-1]
        b = b[:-1]
        
    return c[::-1]

### Test cases

a = np.array([
    0.4456796414,
    0.160623141,
    0.137676978,
    0.1032577335])

b = np.array([
    0.006737946999,
    0.033689735,
    0.08422433749,
    0.1403738958])

print(convolve(a, b)) # Expected output: [0.00300297 0.0160971  0.04387608 0.08142421]
    
[4.45679641e-01, 1.60623141e-01, 1.60623141e-01, 1.37676978e-01,
 1.03257734e-01, 6.88384890e-02,


[0.00300297 0.0160971  0.04387608 0.08142421]


A schedule with T intervals can have T states. A state of a schedule at interval T is defined by $p^-_t(i)$, the amount of work left at the end of the state at interval $t-1$ and $v_t(i)$, the arriving amount of work in interval $t$.

In [162]:
"""
A schedule class with a constructor and a method to calculate the system states.
"""
class Schedule:
    def __init__(self, x, d, s):
        self.parameters = dict({'x': x, 'd': d, 's': s})
        self.system = dict({
            'p_min': np.zeros((len(self.parameters['x']), len(self.parameters['s'])), dtype=np.float64),
            'v': np.zeros((len(self.parameters['x']), len(self.parameters['s'])), dtype=np.float64)
        })
        self.system['p_min'][0][0] = 1
        self.system['v'][0] = self.parameters['s'].copy()
        self.system['p_plus'] = self.system['v'].copy()
        self.state = 0 # 0: initial state
        
    def calculate_system_states(self, until = 1):

            while self.state < until:
                """Set state to 1, because state 0 has already been calculated in the constructor."""
                
                self.state += 1
                
                """The probability that the amount of work left in the system equals zero just before state t starts 
                is the probablity that the total amount work in state t-1 was less than or equal to the interval length d."""
                
                self.system['p_min'][self.state][0] = np.sum(self.system['p_plus'][self.state-1][:(self.parameters['d'] + 1)])
                
                """The probability that the amount of work left in the system equals i just before state t starts 
                is the probablity that the total amount work in state t-1 exceeded the interval length d with amount i."""
                
                self.system['p_min'][self.state][1:(-1*self.parameters['d'])] = self.system['p_plus'][self.state-1][(self.parameters['d'] + 1):]
                
                """Assuming every state has 1 patient (just for testing), service time distribution will be the same in every state."""
                
                self.system['v'][self.state] = self.system['v'][self.state - 1] #
                
                """The probablitity that the amount of work left in the system equals i just before state t ends equals the convoloation of the probability that the amount of work left in the system equals i just before state t starts and the service time distribution."""
                
                self.system['p_plus'][self.state] = convolve(self.system['p_min'][self.state], self.system['v'][self.state])


In [163]:
"""Simple test case"""

schedule = Schedule(x=np.array([1, 0, 2], dtype=np.float64), d=3, s=np.array(
    [0.1, 0.2, 0.3, 0.2, 0.15, 0.05], dtype=np.float64))
schedule.calculate_system_states(until=2)
schedule.system

{'p_min': array([[1.    , 0.    , 0.    , 0.    , 0.    , 0.    ],
        [0.8   , 0.15  , 0.05  , 0.    , 0.    , 0.    ],
        [0.745 , 0.165 , 0.0725, 0.    , 0.    , 0.    ]]),
 'v': array([[0.1 , 0.2 , 0.3 , 0.2 , 0.15, 0.05],
        [0.1 , 0.2 , 0.3 , 0.2 , 0.15, 0.05],
        [0.1 , 0.2 , 0.3 , 0.2 , 0.15, 0.05]]),
 'p_plus': array([[0.1    , 0.2    , 0.3    , 0.2    , 0.15   , 0.05   ],
        [0.08   , 0.175  , 0.275  , 0.215  , 0.165  , 0.0725 ],
        [0.0745 , 0.1655 , 0.26375, 0.213  , 0.1665 , 0.0765 ]])}

In [164]:
""""Test case with validation in spreadsheet"""
x = np.array([1, 0, 1, 0, 1], dtype=np.float64)
d = 5
states = range(0, 30)
l = 6
poisson.pmf(0, l)
s = np.array([poisson.pmf(i, l) for i in states], dtype=np.float64)
schedule = Schedule(x=x, d=d, s=s)
schedule.system
schedule.calculate_system_states(until=2)
schedule.system


{'p_min': array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00],
        [4.45679641e-01, 1.60623141e-01, 1.37676978e-01, 1.03257734e-01,
         6.88384890e-02, 4.13030934e-02, 2.25289600e-02, 1.12644800e-02,
         5.19899078e-03, 2.22813891e-03, 8.91255562e-04, 3.34220836e-04,
         1.17960295e-04, 3.93200983e-05, 1.24168732e-05, 3.72506195e-06,
         1.06430341e-06, 2.90264567e-07, 7.57211915e-08, 1.89302979e-08,
         4.54327149e-09, 1.04844727e-09, 2.32988281e-10, 4.99260603e-11,


In [146]:
np.array([2.47875218e-03, 1.48725131e-02, 4.46175392e-02, 8.92350784e-02,
          1.33852618e-01, 1.60623141e-01]).sum()


0.44567964188